# Clustering movies

In [12]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from datetime import datetime
from imdb import IMDb

In [2]:
movies = pd.read_csv('movies.csv',encoding='latin1')
movies

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986
5,6000000.0,Hemdale,UK,Oliver Stone,Drama,138530565.0,Platoon,R,1987-02-06,120,8.1,Charlie Sheen,317585,Oliver Stone,1986
6,25000000.0,Henson Associates (HA),UK,Jim Henson,Adventure,12729917.0,Labyrinth,PG,1986-06-27,101,7.4,David Bowie,102879,Dennis Lee,1986
7,6000000.0,De Laurentiis Entertainment Group (DEG),USA,David Lynch,Drama,8551228.0,Blue Velvet,R,1986-10-23,120,7.8,Isabella Rossellini,146768,David Lynch,1986
8,9000000.0,Paramount Pictures,USA,Howard Deutch,Comedy,40471663.0,Pretty in Pink,PG-13,1986-02-28,96,6.8,Molly Ringwald,60565,John Hughes,1986
9,15000000.0,SLM Production Group,USA,David Cronenberg,Drama,40456565.0,The Fly,R,1986-08-15,96,7.5,Jeff Goldblum,129698,George Langelaan,1986


In [3]:
movies.isna().any()

budget      False
company     False
country     False
director    False
genre       False
gross       False
name        False
rating      False
released    False
runtime     False
score       False
star        False
votes       False
writer      False
year        False
dtype: bool

In [4]:
movies.describe(include=object)

,company,country,director,genre,name,rating,released,star,writer
count,6820,6820,6820,6820,6820,6820,6820,6820,6820
unique,2179,57,2759,17,6731,13,2403,2504,4199
top,Universal Pictures,USA,Woody Allen,Comedy,Hamlet,R,1991-10-04,Nicolas Cage,Woody Allen
freq,302,4872,33,2080,3,3392,10,42,32


In [5]:
movies.describe()

,budget,gross,runtime,score,votes,year
count,6.820000e+03,6.820000e+03,6820.00000,6820.000000,6.820000e+03,6820.000000
mean,2.458113e+07,3.349783e+07,106.55132,6.374897,7.121952e+04,2001.000293
std,3.702254e+07,5.819760e+07,18.02818,1.003142,1.305176e+05,8.944501
min,0.000000e+00,7.000000e+01,50.00000,1.500000,2.700000e+01,1986.000000
25%,0.000000e+00,1.515839e+06,95.00000,5.800000,7.665250e+03,1993.000000
50%,1.100000e+07,1.213568e+07,102.00000,6.400000,2.589250e+04,2001.000000
75%,3.200000e+07,4.006534e+07,115.00000,7.100000,7.581225e+04,2009.000000
max,3.000000e+08,9.366622e+08,366.00000,9.300000,1.861666e+06,2016.000000


In [6]:
movies.dtypes

budget      float64
company      object
country      object
director     object
genre        object
gross       float64
name         object
rating       object
released     object
runtime       int64
score       float64
star         object
votes         int64
writer       object
year          int64
dtype: object

In [7]:
movies.released.apply(lambda x: len(x)).value_counts()

10    6756
7       57
4        7
Name: released, dtype: int64

In [8]:
small_dates = movies.loc[movies.released.str.len()<10].released.tolist()
redone = []
for i in small_dates:
    if len(i)==4:
        redone.append(i+'-01-01')
    else:
        redone.append(i+'-01')
movies.loc[movies.released.str.len()<10,['released']] = redone

In [9]:
def release_to_days(X,since=datetime(1986,1,1)):
    to = datetime.strptime(X,'%Y-%m-%d')
    return (to-since).days

movies['days_released']=movies.released.apply(lambda x: release_to_days(x))

movies.drop(['released'],axis=1,inplace=True)

This block will take a long time to run because there is a request limit to IMDB. I will include movies dataset with plots.

In [ ]:
def get_plot(X,search=IMDb()):
    ms = search.search_movie(X['name'])
    
    for i,val in enumerate(ms):
        
        movieID = ms[i].movieID
        movie = search.get_movie(movieID)
        
        if movie.get('year') == X.year:
            return movie.get('plot outline')
        
movies['plot'] = movies.apply(lambda x: get_plot(x),axis=1)

In [56]:
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import features_generator
text_encoding = Pipeline([
        ('count_vect',CountVectorizer(stop_words='english',analyzer='word')),
        ('tfidf',TfidfTransformer())
])
cols = ['director','company','country','genre','rating','star','writer']
for i in cols:
    text

<6820x3303 sparse matrix of type '<class 'numpy.float64'>'
	with 13982 stored elements in Compressed Sparse Row format>

In [21]:
final_transformer.fit_transform(movies[['director','company','country','genre','rating','star','writer']])

TypeError: no supported conversion for types: (dtype('O'),)